In [1]:
from typing import Optional, Union, Tuple

import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)
# import tensorflow_probability as tfp
from keras import ops
# import tensorflow_addons as tfa
# from keras.activations import gelu as keras_gelu

import pandas as pd
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
import keras
from keras import layers

@tf.function
def identity(x):
    return x

ModuleNotFoundError: No module named 'numpy'

In [3]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf
import numpy as np

SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [1]:
from keras import ops


SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)  

num_samples = 1000

data = {}
for i in range(1, 42):
    col_name = f"Feature_{i}"
    data[col_name] = np.random.rand(num_samples)  

df = pd.DataFrame(data)
df['Target'] = np.random.randint(0, 2, size=num_samples) 

print(df.head())
feature_columns = list(df.columns[:-1])  

print(f"Số lượng mẫu: {len(df)}, Số lượng cột: {len(df.columns)}")



   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
0   0.374540   0.185133   0.261706   0.672703   0.571996   0.393636   
1   0.950714   0.541901   0.246979   0.796681   0.805432   0.473436   
2   0.731994   0.872946   0.906255   0.250468   0.760161   0.854547   
3   0.598658   0.732225   0.249546   0.624874   0.153900   0.340004   
4   0.156019   0.806561   0.271950   0.571746   0.149249   0.869650   

   Feature_7  Feature_8  Feature_9  Feature_10  Feature_11  Feature_12  Target  
0   0.648257   0.038799   0.720268    0.913578    0.373641    0.533031       1  
1   0.172386   0.186773   0.687283    0.525360    0.332912    0.137899       1  
2   0.872395   0.831246   0.095754    0.724910    0.176154    0.591243       0  
3   0.613116   0.766768   0.922572    0.436048    0.607267    0.314786       0  
4   0.157204   0.350643   0.568472    0.630035    0.476624    0.052349       0  
Số lượng mẫu: 1000, Số lượng cột: 13


In [3]:
class GLUBlock(tf.keras.layers.Layer):
    def __init__(self, units: Optional[int] = None,
                #  virtual_batch_size: Optional[int] = 128, 
                 momentum: Optional[float] = 0.02):
        super(GLUBlock, self).__init__()
        self.units = units
        # self.virtual_batch_size = virtual_batch_size
        self.momentum = momentum
        
    def build(self, input_shape: tf.TensorShape):
        if self.units is None:
            self.units = input_shape[-1]
            
        self.fc_outout = tf.keras.layers.Dense(self.units, 
                                               use_bias=False)
        self.bn_outout = tf.keras.layers.BatchNormalization(
                                                            momentum=self.momentum)
        
        self.fc_gate = tf.keras.layers.Dense(self.units, 
                                             use_bias=False)
        self.bn_gate = tf.keras.layers.BatchNormalization( 
                                                          momentum=self.momentum)
        
    def call(self, inputs: Union[tf.Tensor, np.ndarray], training: Optional[bool] = None):
        output = self.bn_outout(self.fc_outout(inputs), 
                                training=training)
        gate = self.bn_gate(self.fc_gate(inputs), 
                            training=training)
    
        return output * tf.keras.activations.sigmoid(gate) # GLU
    
class FeatureTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, units: Optional[int] = None,
                 momentum: Optional[float] = 0.02, skip=False):
        super(FeatureTransformerBlock, self).__init__()
        self.units = units
        # self.virtual_batch_size = virtual_batch_size
        self.momentum = momentum
        self.skip = skip
        
    def build(self, input_shape: tf.TensorShape):
        if self.units is None:
            self.units = input_shape[-1]
        
        self.initial = GLUBlock(units = self.units, 
                                # virtual_batch_size=self.virtual_batch_size, 
                                momentum=self.momentum)
        self.residual =  GLUBlock(units = self.units, 
                                #   virtual_batch_size=self.virtual_batch_size, 
                                  momentum=self.momentum)
        
    def call(self, inputs: Union[tf.Tensor, np.ndarray], training: Optional[bool] = None):
        initial = self.initial(inputs, training=training)
        
        if self.skip == True:
            initial += inputs

        residual = self.residual(initial, training=training) # skip
        
        return (initial + residual) * np.sqrt(0.5)

class AttentiveTransformer(tf.keras.layers.Layer):
    def __init__(self, units: Optional[int] = None, 
                #  virtual_batch_size: Optional[int] = 128, 
                 momentum: Optional[float] = 0.02):
        super(AttentiveTransformer, self).__init__()
        self.units = units
        # self.virtual_batch_size = virtual_batch_size
        self.momentum = momentum
        
    def build(self, input_shape: tf.TensorShape):
        if self.units is None:
            self.units = input_shape[-1]
            
        self.fc = tf.keras.layers.Dense(self.units, 
                                        use_bias=False)
        self.bn = tf.keras.layers.BatchNormalization(
            # virtual_batch_size=self.virtual_batch_size, 
                                                     momentum=self.momentum)
        
    def call(self, inputs: Union[tf.Tensor, np.ndarray], priors: Optional[Union[tf.Tensor, np.ndarray]] = None, training: Optional[bool] = None) -> tf.Tensor:
        feature = self.bn(self.fc(inputs), 
                          training=training)
        if priors is None:
            output = feature
        else:
            output = feature * priors
        
        return keras.layers.sparsemax(output)

class TabNetStep(tf.keras.layers.Layer):
    def __init__(self, units: Optional[int] = None,
                 momentum: Optional[float] =0.02):
        super(TabNetStep, self).__init__()
        self.units = units
        # self.virtual_batch_size = virtual_batch_size
        self.momentum = momentum
        
    def build(self, input_shape: tf.TensorShape):
        if self.units is None:
            self.units = input_shape[-1]
        
        self.unique = FeatureTransformerBlock(units = self.units, 
                                            #   virtual_batch_size=self.virtual_batch_size, 
                                              momentum=self.momentum,
                                              skip=True)
        self.attention = AttentiveTransformer(units = input_shape[-1], 
                                            #   virtual_batch_size=self.virtual_batch_size, 
                                              momentum=self.momentum)
        
    def call(self, inputs, shared, priors, training=None) -> Tuple[tf.Tensor]:  
        split = self.unique(shared, training=training)
        keys = self.attention(split, priors, training=training)
        masked = keys * inputs
        
        return split, masked, keys

In [4]:
class TabNetEncoder(tf.keras.layers.Layer):
    def __init__(self, units: int =1, 
                 n_steps: int = 3, 
                 n_features: int = 8,
                 outputs: int = 1, 
                 gamma: float = 1.3,
                 epsilon: float = 1e-8, 
                 sparsity: float = 1e-5, 
                #  virtual_batch_size: Optional[int]=128, 
                 momentum: Optional[float] =0.02):
        super(TabNetEncoder, self).__init__()
        
        self.units = units
        self.n_steps = n_steps
        self.n_features = n_features
        # self.virtual_batch_size = virtual_batch_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.momentum = momentum
        self.sparsity = sparsity
        
    def build(self, input_shape: tf.TensorShape):            
        self.bn = tf.keras.layers.BatchNormalization(
            # virtual_batch_size=self.virtual_batch_size, 
                                                     momentum=self.momentum)
        self.shared_block = FeatureTransformerBlock(units = self.n_features, 
                                                    # virtual_batch_size=self.virtual_batch_size, 
                                                    momentum=self.momentum)        
        self.initial_step = TabNetStep(units = self.n_features, 
                                    #    virtual_batch_size=self.virtual_batch_size, 
                                       momentum=self.momentum)
        self.steps = [TabNetStep(units = self.n_features, 
                                #  virtual_batch_size=self.virtual_batch_size, 
                                 momentum=self.momentum) for _ in range(self.n_steps)]
        self.final = tf.keras.layers.Dense(units = self.units, 
                                           use_bias=False)
    

    def call(self, X: Union[tf.Tensor, np.ndarray], training: Optional[bool] = None) -> Tuple[tf.Tensor]:        
        entropy_loss = 0.
        encoded = 0.
        output = 0.
        importance = 0.
        prior = tf.reduce_mean(tf.ones_like(X), axis=0)
        
        B = prior * self.bn(X, training=training)
        shared = self.shared_block(B, training=training)
        _, masked, keys = self.initial_step(B, shared, prior, training=training)

        for step in self.steps:
            entropy_loss += tf.reduce_mean(tf.reduce_sum(-keys * tf.math.log(keys + self.epsilon), axis=-1)) / tf.cast(self.n_steps, tf.float32)
            prior *= (self.gamma - tf.reduce_mean(keys, axis=0))
            importance += keys
            
            shared = self.shared_block(masked, training=training)
            split, masked, keys = step(B, shared, prior, training=training)
            features = tf.keras.activations.relu(split)
            
            output += features
            encoded += split
            
        self.add_loss(self.sparsity * entropy_loss)
          
        prediction = self.final(output)
        return prediction, encoded, importance

In [5]:
class TabNetDecoder(tf.keras.layers.Layer):
    def __init__(self, units=1, 
                 n_steps = 3, 
                 n_features = 8,
                 outputs = 1, 
                 gamma = 1.3,
                 epsilon = 1e-8, 
                 sparsity = 1e-5, 
                #  virtual_batch_size=128, 
                 momentum=0.02):
        super(TabNetDecoder, self).__init__()
        
        self.units = units
        self.n_steps = n_steps
        self.n_features = n_features
        # self.virtual_batch_size = virtual_batch_size
        self.momentum = momentum
        
    def build(self, input_shape: tf.TensorShape):
        self.shared_block = FeatureTransformerBlock(units = self.n_features, 
                                                    # virtual_batch_size=self.virtual_batch_size, 
                                                    momentum=self.momentum)
        self.steps = [FeatureTransformerBlock(units = self.n_features,
                                            #   virtual_batch_size=self.virtual_batch_size, 
                                              momentum=self.momentum) for _ in range(self.n_steps)]
        self.fc = [tf.keras.layers.Dense(units = self.units) for _ in range(self.n_steps)]
    

    def call(self, X: Union[tf.Tensor, np.ndarray], training: Optional[bool] = None) -> tf.Tensor:
        decoded = 0.
        
        for ftb, fc in zip(self.steps, self.fc):
            shared = self.shared_block(X, training=training)
            feature = ftb(shared, training=training)
            output = fc(feature)
            
            decoded += output
        return decoded

In [6]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

### Encoder VAE

In [42]:
class VAE_Encoder(tf.keras.layers.Layer):
    def __init__(self, 
                 outputs: int = 1, 
                 inputs: int = 12,
                 n_steps: int = 3, 
                 n_features: int = 12,
                 gamma: float = 1.3, 
                 epsilon: float = 1e-8, 
                 sparsity: float = 1e-5, 
                #  virtual_batch_size: Optional[int] = None, 
                 momentum: Optional[float] = 0.02, 
                 mlp_units=None, 
                 latent_dim=64, 
                 name='vae_encoder', 
                 **kwargs):
        
        super(VAE_Encoder, self).__init__(name=name, **kwargs)
        self.feature = tf.keras.layers.Lambda(lambda x: x)
        # if feature_column is None:
        # else:
        #     self.feature = feature_column
        self.tabnet_encoder = TabNetEncoder(units=outputs, 
                                    n_steps=n_steps, 
                                    n_features=n_features,
                                    gamma=gamma, 
                                    epsilon=epsilon, 
                                    sparsity=sparsity,
                                    # virtual_batch_size=virtual_batch_size, 
                                    momentum=momentum)
        if mlp_units:
            self.mlp = tf.keras.Sequential([
                tf.keras.layers.Dense(units, activation='relu') for units in mlp_units
            ])
        self.mean_layer = tf.keras.layers.Dense(latent_dim)
        self.logvar_layer = tf.keras.layers.Dense(latent_dim)

    def call(self, inputs, training=False):
        _, encoded, _ = self.tabnet_encoder(inputs)
        print('ok')
        encoded = self.mlp(encoded, training=training)
        z_mean = self.mean_layer(encoded)
        z_log_var = self.logvar_layer(encoded)
        z = Sampling()([z_mean, z_log_var])
        return z_mean, z_log_var,z


### Decoder VAE

In [43]:
class VAE_Decoder(tf.keras.layers.Layer):
    def __init__(self, 
                 outputs: int = 1, 
                 inputs: int = 12,
                 n_steps: int = 3, 
                 n_features: int = 12,
                 gamma: float = 1.3, 
                 epsilon: float = 1e-8, 
                 sparsity: float = 1e-5, 
                #  virtual_batch_size: Optional[int] = None, 
                 momentum: Optional[float] = 0.02,
                 mlp_units=None, 
                 name='vae_decoder', 
                 **kwargs):
        super(VAE_Decoder, self).__init__(name=name, **kwargs)
        if mlp_units:
            self.mlp = tf.keras.Sequential([
                tf.keras.layers.Dense(units, activation='relu') for units in reversed(mlp_units)
            ])
        self.tabnet_decoder = TabNetDecoder(units=inputs, 
                                     n_steps=n_steps, 
                                     n_features=n_features,
                                    #  virtual_batch_size=virtual_batch_size, 
                                     momentum=momentum)

    def call(self, inputs=64, training=False):
        decoded = self.mlp(inputs, training=training)
        reconstructed = self.tabnet_decoder(decoded, training=training)
        return reconstructed


In [44]:
x_train = df.drop(columns=['Target'])  
y_train = df['Target'] 

In [45]:
x_train

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12
0,0.374540,0.185133,0.261706,0.672703,0.571996,0.393636,0.648257,0.038799,0.720268,0.913578,0.373641,0.533031
1,0.950714,0.541901,0.246979,0.796681,0.805432,0.473436,0.172386,0.186773,0.687283,0.525360,0.332912,0.137899
2,0.731994,0.872946,0.906255,0.250468,0.760161,0.854547,0.872395,0.831246,0.095754,0.724910,0.176154,0.591243
3,0.598658,0.732225,0.249546,0.624874,0.153900,0.340004,0.613116,0.766768,0.922572,0.436048,0.607267,0.314786
4,0.156019,0.806561,0.271950,0.571746,0.149249,0.869650,0.157204,0.350643,0.568472,0.630035,0.476624,0.052349
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.091582,0.656955,0.865296,0.442107,0.897397,0.684435,0.579737,0.273160,0.514929,0.857656,0.359445,0.004644
996,0.917314,0.956615,0.157273,0.334401,0.119381,0.503220,0.667037,0.209519,0.165370,0.897509,0.926708,0.873669
997,0.136819,0.068958,0.309788,0.394572,0.327843,0.765149,0.911577,0.455329,0.977713,0.946708,0.698657,0.307325
998,0.950237,0.057055,0.290046,0.529941,0.815745,0.485291,0.345620,0.908438,0.107925,0.397488,0.257177,0.938589


In [49]:
print(x_train.shape)

(1000, 12)


In [60]:
class VAE_Tabnet_MLPS(tf.keras.Model):
    def __init__(self, encoder, decoder, classifier, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.feature = tf.keras.layers.Lambda(lambda x: x)
        self.classifier = classifier
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        self.classification_loss_tracker = tf.keras.metrics.Mean(name="classification_loss")
        self.accuracy_tracker = tf.keras.metrics.BinaryAccuracy(name="accuracy")
        self.bn = tf.keras.layers.BatchNormalization(momentum=0.02)
        self.do = tf.keras.layers.Dropout(0.25)

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.classification_loss_tracker,
            self.accuracy_tracker,
        ]
    
        

    # def call(self, X: Union[tf.Tensor, np.ndarray], training: Optional[bool] = None) -> tf.Tensor:
    #     X = self.feature(X)
    #     print(X)
    #     X = self.bn(X, training=training)
        
    #     M = self.do(tf.ones_like(X), training=training)
    #     D = X * M
    #     print(X.shape)
        
    #     output, encoded, importance = self.encoder(D)
    #     prediction = tf.keras.activations.sigmoid(output)        
        
    #     T = X * (1 - M)
    #     reconstruction = self.decoder(encoded)
        
    #     loss = tf.reduce_mean(tf.where(M != 0., tf.square(T - reconstruction), tf.zeros_like(reconstruction)))
    #     self.add_loss(loss)
        
    #     return prediction

    def train_step(self, inputs): 
        # print(inputs.shape)
        x_train,y_train=inputs
        print(x_train.shape,y_train.shape)

        # # x_train, y_train = data 
        # # print(f"x_train shape: {x_train.shape}")
        # X = self.feature(X)[0]
        # print(X.shape)
        # X = self.bn(X, training=training)
        # print(X.shape)
        
        # M = self.do(tf.ones_like(X), training=training)
        # # D = X * M

        with tf.GradientTape() as tape:
            z, z_mean, z_log_var = self.encoder(x_train)
            reconstruction = self.decoder(z)
            classification_output = self.classifier(z)
            
            reconstruction_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(x_train, reconstruction)
            )

            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            )
            
            classification_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(y_train, classification_output)
            )

            total_loss = reconstruction_loss + kl_loss + classification_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.classification_loss_tracker.update_state(classification_loss)
        self.accuracy_tracker.update_state(y_train, classification_output)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "classification_loss": self.classification_loss_tracker.result(),
            "accuracy": self.accuracy_tracker.result(),
        }


In [61]:
def build_classifier(input_dim, num_classes):
    inputs = Input(shape=(input_dim,))
    print(inputs.shape)
    x = Dense(32, activation='relu')(inputs)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)  # Sử dụng softmax cho phân loại đa lớp
    classifier = Model(inputs=inputs, outputs=outputs)
    classifier.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy')
    return classifier
classifier= build_classifier(64,1)

(None, 64)


In [63]:
latent_dim = 64
# tabnet_encoder = TabNetEncoder(units=latent_dim, n_steps=3, n_features=12)
# tabnet_decoder = TabNetDecoder(units=12, n_steps=3, n_features=12)

vae_encoder = VAE_Encoder(mlp_units=[64, 32], latent_dim=64, outputs=64, inputs=12)
vae_decoder = VAE_Decoder(mlp_units=[32, 64], outputs=12, inputs=latent_dim)
vae = VAE_Tabnet_MLPS(encoder=vae_encoder, decoder=vae_decoder, classifier=classifier)

vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy')
print(x_train.shape)
vae.fit(x_train, y_train, epochs=50)


(1000, 12)
Epoch 1/50
(32, 12) (32, 1)


c:\Users\PC\miniconda3\envs\vae\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\PC\miniconda3\envs\vae\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


AttributeError: Exception encountered when calling AttentiveTransformer.call().

[1mmodule 'keras.api.layers' has no attribute 'sparsemax'[0m

Arguments received by AttentiveTransformer.call():
  • inputs=tf.Tensor(shape=(32, 12), dtype=float32)
  • priors=tf.Tensor(shape=(12,), dtype=float32)
  • training=False